In [40]:
## run t test on the harmonic features (entropy of the spectrum power histogram)

from scipy import stats
import pandas as pd
import numpy as np
import os
import re
import glob
import seaborn as sns
import matplotlib.pyplot as plt

input_dir = 'adni_out04_power_histogram_and_entropy'
atlas_file = '/home/xin/Downloads/Atlases/power264_roi_lut.csv'

output_dir = 'adni_out05_ttest'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    

In [32]:
## do ttest and calculate raw p values:

data_files = glob.glob(input_dir + '/adni_out04_entropy_freq_*.csv')
data_files.sort()

# all_result = []
print_group_size = True

comparision = [('CN', 'AD'), ('CN', 'EMCI'), ('CN', 'LMCI'), ('CN', 'SMC')]
result_pvalue_allfreq = []
result_stat_allfreq = []

for f in data_files:
    
    result_pvalue = {comp: [] for comp in comparision}
    result_stat = {comp: [] for comp in comparision}
    
    #print(f)
    freq = re.search('(.*)_freq_(.*).csv', f).group(2)
    data = pd.read_csv(f, index_col = 0)
    data.dropna(subset=['DX'], inplace = True)
    
    if print_group_size:
        print(data.groupby(['PTGENDER', 'DX']).size())
        print_group_size = False
    
    for feature in range(264):
        
        for comp in comparision:
            # select data with ad/lmci and cn:    
            group1 = data.loc[data['DX'].isin([comp[0]]), str(feature)].values
            group2 = data.loc[data['DX'].isin([comp[1]]), str(feature)].values

            ttest = stats.ttest_ind(group1, group2, equal_var = False)
            
            result_pvalue[comp].append(ttest.pvalue)
            result_stat[comp].append(ttest.statistic)

    result_pvalue_allfreq.append(result_pvalue)
    result_stat_allfreq.append(result_stat)

print('finished')


PTGENDER  DX  
Female    AD      17
          CN      50
          EMCI    43
          LMCI    28
          SMC     23
Male      AD      16
          CN      44
          EMCI    42
          LMCI    31
          SMC     17
dtype: int64
finished


In [33]:
data_files

['adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_00.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_01.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_02.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_03.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_04.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_05.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_06.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_07.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_08.csv',
 'adni_out04_power_histogram_and_entropy/adni_out04_entropy_freq_09.csv']

In [64]:
## box plot and FDR correction on p values:

# add p value annotation:
def add_annot(x1, x2, y, h, p, roiname, ax, color = 'k'):
    roiname = roiname.replace('?', '')
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1, c=color)
    if p>=.001:
        txt = roiname + ": ${p}$=%.3f" % p
    else:
        txt = roiname + ": ${p}$<.001"

    #print(txt)
    ax.text((x1+x2)*.5, y+h, txt, ha='center', va='bottom', color=color, size = 10)
    
# https://stackoverflow.com/questions/7450957/how-to-implement-rs-p-adjust-in-python/33532498#33532498
def p_adjust_bh(p):
    """Benjamini-Hochberg p-value correction for multiple hypothesis testing."""
    p = np.asfarray(p)
    by_descend = p.argsort()[::-1]
    by_orig = by_descend.argsort()
    steps = float(len(p)) / np.arange(len(p), 0, -1)
    q = np.minimum(1, np.minimum.accumulate(steps * p[by_descend]))
    return q[by_orig]
    
    
atlas_info = pd.read_csv(atlas_file, header = None)

for i, f in enumerate(data_files):
    
    result_pvalue = result_pvalue_allfreq[i]
    
    # freq should equal to i:
    freq = re.search('(.*)_freq_(.*).csv', f).group(2)
    data = pd.read_csv(f, index_col = 0)
    data.dropna(subset=['DX'], inplace = True)
    for comp in comparision:
        
        # FDR correction for p values:
        pvalue_fdr = p_adjust_bh(result_pvalue[comp])
        
        for feature, pvalue in enumerate(pvalue_fdr):
            
            roi_name = atlas_info.loc[feature, 0]        
            # select data with ad/lmci and cn:    
            group1 = data.loc[data['DX'].isin([comp[0]]), str(feature)].values
            group2 = data.loc[data['DX'].isin([comp[1]]), str(feature)].values
            
            if pvalue < .05:
            
                ## box plot for each brain region:
                a4_dims = (4.27, 4.27)
                fig, ax1 = plt.subplots(figsize=a4_dims)

                plot_data = pd.concat([pd.DataFrame({'entropy': group1, 'group': comp[0]}), 
                                       pd.DataFrame({'entropy': group2, 'group': comp[1]})], axis = 0)

                xvals = np.unique(plot_data.group)
                positions = range(len(xvals))
                plt.boxplot([plot_data[plot_data.group == xi].entropy for xi in xvals[::-1]],
                            positions=positions, showfliers=False,
                            boxprops={'facecolor': 'none'}, medianprops={'color': 'black'}, patch_artist=True,
                            widths=[0.4, 0.3])

                # plt.xticks(positions, xvals) # not needed anymore, as the xticks are set by the swarmplot
                sns.swarmplot(x = 'group', y = 'entropy', data = plot_data)
                
                ax1.spines.right.set_visible(False)
                ax1.spines.top.set_visible(False)
                
                pos_y = np.max(np.hstack((group1, group2))) * 1.015
                pos_x1, pos_x2 = 0, 1
                add_annot(pos_x1, pos_x2, pos_y, .05, pvalue, roi_name, ax1, color = 'k')
                
                figure_name = '/box_plot_' + 'freq_'+str(freq)+'_roi_'+str(feature)+'_' \
                                + roi_name.replace('/', '|') + '{:.3f}'.format(pvalue)+'_'+comp[0]+'_'+comp[1]+'.png'
                plt.savefig(output_dir + figure_name,
                            transparent = True)
                plt.clf()

        # break
    # break

print('finished')


<ipython-input-64-97f5688b4e11>:52: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 = plt.subplots(figsize=a4_dims)
/home/xin/miniconda3/lib/python3.8/site-packages/seaborn/categorical.py:1296: UserWarning: 7.4% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


finished


<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

...

<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

<Figure size 307.44x307.44 with 0 Axes>

In [63]:
for comp in comparision:
        res = pd.DataFrame([result_pvalue[comp] for result_pvalue in result_pvalue_allfreq])
        res = res.transpose()
        res.index = ['roi' + str(i) for i in range(1, 265)]
        res.columns = ['freq' + str(i) for i in range(1, 11)]

        res.to_csv(output_dir + '/lasso_pvalue_' + comp[0] + '_' + comp[1] + '.csv')

        res = pd.DataFrame([result_stat[comp] for result_stat in result_stat_allfreq])
        res = res.transpose()
        res.index = ['roi' + str(i) for i in range(1, 265)]
        res.columns = ['freq' + str(i) for i in range(1, 11)]

        res.to_csv(output_dir + '/lasso_stats_' + comp[0] + '_' + comp[1] + '.csv')

In [ ]:
res